In [4]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import *
from pyspark.sql.functions import to_date, year, month, dayofmonth, dayofweek, hour
from pyspark.sql.functions import round, count, mean, min, max, stddev, countDistinct, col, when, datediff, desc, avg
from pyspark.sql import Window
import math

spark = SparkSession.builder \
    .appName("DivvyBikes_Save_AV") \
    .master("local[4]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.default.parallelism", "200") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.minPartitionSize", "128m") \
    .config("spark.sql.adaptive.advisoryPartitionSizeInBytes", "128m") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.3") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .config("spark.sql.broadcastTimeout", "1800") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "512m") \
    .config("spark.kryoserializer.buffer", "128m") \
    .config("spark.executor.extraJavaOptions", 
           "-XX:+UseG1GC -XX:+UseCompressedOops -XX:+UseStringDeduplication -XX:MaxGCPauseMillis=200") \
    .config("spark.driver.extraJavaOptions", 
           "-XX:+UseG1GC -XX:+UseCompressedOops -XX:MaxGCPauseMillis=200") \
    .config("spark.network.timeout", "1200s") \
    .config("spark.executor.heartbeatInterval", "120s") \
    .config("spark.locality.wait", "0") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2") \
    .config("spark.hadoop.parquet.enable.summary-metadata", "false") \
    .config("spark.sql.parquet.mergeSchema", "false") \
    .config("spark.sql.parquet.filterPushdown", "true") \
    .config("spark.sql.hive.convertMetastoreParquet", "false") \
    .config("spark.sql.inMemoryColumnarStorage.compressed", "true") \
    .config("spark.sql.inMemoryColumnarStorage.batchSize", "10000") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false") \
    .config("spark.hadoop.fs.s3a.fast.upload", "true") \
    .config("spark.hadoop.fs.s3a.fast.upload.buffer", "disk") \
    .config("spark.hadoop.mapreduce.output.fileoutputformat.compress", "true") \
    .config("spark.hadoop.mapreduce.output.fileoutputformat.compress.codec", "org.apache.hadoop.io.compress.GzipCodec") \
    .getOrCreate()

# Установим уровень логирования
spark.sparkContext.setLogLevel("ERROR")

print("=" * 80)
print("SPARK СЕССИЯ СОЗДАНА ДЛЯ ОБРАБОТКИ 31 МЛН СТРОК")
print("=" * 80)

# ============================================
# СХЕМА ДАННЫХ
# ============================================

schema = StructType([
    StructField("ride_id", StringType(), True),
    StructField("rideable_type", StringType(), True),
    StructField("started_at", TimestampType(), True),
    StructField("ended_at", TimestampType(), True),
    StructField("start_station_name", StringType(), True),
    StructField("start_station_id", StringType(), True),
    StructField("end_station_name", StringType(), True),
    StructField("end_station_id", StringType(), True),
    StructField("start_lat", DoubleType(), True),
    StructField("start_lng", DoubleType(), True),
    StructField("end_lat", DoubleType(), True),
    StructField("end_lng", DoubleType(), True),
    StructField("member_casual", StringType(), True)
])

# ============================================
# ЗАГРУЗКА ДАННЫХ
# ============================================

print("\n" + "="*80)
print("НАЧАЛО ЗАГРУЗКИ ДАННЫХ ИЗ output.csv")
print("="*80)

try:
    # Добавляем больше опций для обработки больших файлов
    df = spark.read \
        .option("header", "true") \
        .schema(schema) \
        .option("timestampFormat", "yyyy-MM-dd HH:mm:ss") \
        .option("encoding", "UTF-8") \
        .option("mode", "DROPMALFORMED") \
        .option("inferSchema", "false") \
        .option("recursiveFileLookup", "true") \
        .option("maxColumns", "200") \
        .option("nullValue", "") \
        .option("nanValue", "") \
        .csv("output.csv")
    
    # Проверяем загрузку
    initial_count = df.count()
    print(f"✓ УСПЕШНО ЗАГРУЖЕНО: {initial_count:,} строк")
    print(f"✓ КОЛИЧЕСТВО СТОЛБЦОВ: {len(df.columns)}")
    print(f"✓ РАЗМЕР ДАННЫХ В ПАМЯТИ: ~{initial_count * len(df.columns) * 8 / (1024**3):.2f} GB (примерно)")
    
except Exception as e:
    print(f"✗ ОШИБКА ПРИ ЗАГРУЗКЕ: {str(e)}")
    raise

# ============================================
# ПРЕДВАРИТЕЛЬНАЯ ОПТИМИЗАЦИЯ
# ============================================

# Перераспределяем данные для лучшей параллельности
df = df.repartition(200)

# ============================================
# Создание признаков
# ============================================

print("\n" + "="*80)
print("СОЗДАНИЕ ПРИЗНАКОВ И ПРЕОБРАЗОВАНИЯ")
print("="*80)

df_with_features = df.withColumn("ride_date", to_date(col("started_at"))) \
    .withColumn("ride_year", year(col("started_at"))) \
    .withColumn("ride_month", month(col("started_at"))) \
    .withColumn("ride_day", dayofmonth(col("started_at"))) \
    .withColumn("ride_dayofweek", dayofweek(col("started_at"))) \
    .withColumn("ride_hour", hour(col("started_at"))) \
    .withColumn("ride_duration_seconds", 
               (col("ended_at").cast("long") - col("started_at").cast("long"))) \
    .withColumn("ride_duration_minutes", 
               (col("ended_at").cast("long") - col("started_at").cast("long")) / 60.0)

# Функция для расчета расстояния (формула Гаверсинуса)
def haversine_distance_spark(lat1, lon1, lat2, lon2):
    """Расчет расстояния с использованием функций Spark"""
    R = 6371  # Радиус Земли в км
    
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = sin(dlat/2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon/2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return R * c

# Добавляем географические признаки
df_with_geo_features = df_with_features \
    .withColumn("distance_km", haversine_distance_spark(
        col("start_lat"), col("start_lng"), col("end_lat"), col("end_lng"))) \
    .withColumn("lat_grid", round(col("start_lat"), 2)) \
    .withColumn("lng_grid", round(col("start_lng"), 2)) \
    .withColumn("is_weekend", 
                when((col("ride_dayofweek") == 1) | 
                     (col("ride_dayofweek") == 7), True).otherwise(False)) \
    .withColumn("lat_diff", abs(col("end_lat") - col("start_lat"))) \
    .withColumn("lng_diff", abs(col("end_lng") - col("start_lng"))) \
    .withColumn("coord_diff", col("lat_diff") + col("lng_diff")) \
    .withColumn("same_station", 
                when(col("start_station_name") == col("end_station_name"), True).otherwise(False))

# ============================================
# ФИЛЬТРАЦИЯ ДАННЫХ
# ============================================

print("\n" + "="*80)
print("ПРИМЕНЕНИЕ ФИЛЬТРОВ ДЛЯ ОЧИСТКИ ДАННЫХ")
print("="*80)

# Сначала делаем базовые проверки
initial_count = df_with_geo_features.count()
print(f"Начальное количество строк: {initial_count:,}")

# Применяем фильтры
filtered_simple = df_with_geo_features.filter(
    # 1. Базовая валидация
    (col("ride_id").isNotNull()) &
    (col("started_at").isNotNull()) &
    (col("ended_at").isNotNull()) &
    (col("member_casual").isNotNull()) &
    (col("rideable_type").isNotNull()) &
    
    # 2. Даты
    (col("ended_at") > col("started_at")) &
    
    # 3. Длительность
    (col("ride_duration_minutes") >= 0) &
    (col("ride_duration_minutes") <= 1440) &
    
    # 4. Координаты (Чикаго)
    (col("start_lat").between(41.6, 42.1)) &
    (col("start_lng").between(-87.95, -87.5)) &
    (col("end_lat").between(41.6, 42.1)) &
    (col("end_lng").between(-87.95, -87.5)) &
    
    # 5. Не нулевые координаты
    (col("start_lat") != 0) &
    (col("start_lng") != 0) &
    (col("end_lat") != 0) &
    (col("end_lng") != 0) &
    
    # 6. Не тестовые станции
    (~lower(col("start_station_name")).contains("test")) &
    (~lower(col("start_station_name")).contains("hubbard")) &
    (~lower(col("start_station_name")).contains("watson")) &
    (~lower(col("end_station_name")).contains("test")) &
    (~lower(col("end_station_name")).contains("hubbard")) &
    (~lower(col("end_station_name")).contains("watson"))
)

# Удаляем дубликаты
filtered_simple = filtered_simple.dropDuplicates(["ride_id"])

# Проверяем результат фильтрации
filtered_count = filtered_simple.count()
removed_count = initial_count - filtered_count
print(f"✓ После фильтрации: {filtered_count:,} строк")
print(f"✓ Удалено строк: {removed_count:,} ({removed_count/initial_count*100:.2f}%)")

# ============================================
# ДОПОЛНИТЕЛЬНАЯ ОЧИСТКА
# ============================================

print("\n" + "="*80)
print("ДОПОЛНИТЕЛЬНАЯ ОЧИСТКА И ЗАПОЛНЕНИЕ ДАННЫХ")
print("="*80)

# Создаем гео-хеши
filtered_df = filtered_simple.withColumn(
    "start_geo_hash", 
    concat(round(col("start_lat"), 3).cast("string"), lit("_"), 
           round(col("start_lng"), 3).cast("string"))
).withColumn(
    "end_geo_hash", 
    concat(round(col("end_lat"), 3).cast("string"), lit("_"), 
           round(col("end_lng"), 3).cast("string"))
)

# Функция для заполнения пропущенных станций
def fill_missing_stations(df, station_col, geo_hash_col):
    """Заполняет пропущенные названия станций наиболее частыми в той же геозоне"""
    
    # Кэшируем промежуточные данные
    window_spec = Window.partitionBy(geo_hash_col).orderBy(desc("count"))
    
    # Создаем DataFrame с наиболее частыми станциями
    common_stations = df.filter(col(station_col).isNotNull()) \
        .groupBy(geo_hash_col, station_col) \
        .agg(count("*").alias("count")) \
        .withColumn("rank", row_number().over(window_spec)) \
        .filter(col("rank") == 1) \
        .select(geo_hash_col, col(station_col).alias(f"common_{station_col}"))
    
    # Присоединяем наиболее частые станции
    df_filled = df.join(broadcast(common_stations), on=geo_hash_col, how="left") \
        .withColumn(f"{station_col}_clean", 
                   coalesce(col(station_col), col(f"common_{station_col}"))) \
        .drop(f"common_{station_col}")
    
    return df_filled

# Заполняем пропущенные станции
print("Заполнение пропущенных станций...")
filtered_df = fill_missing_stations(filtered_df, "start_station_name", "start_geo_hash")
filtered_df = fill_missing_stations(filtered_df, "end_station_name", "end_geo_hash")

# Заменяем оригинальные колонки
filtered_df = filtered_df \
    .drop("start_station_name", "end_station_name") \
    .withColumnRenamed("start_station_name_clean", "start_station_name") \
    .withColumnRenamed("end_station_name_clean", "end_station_name") \
    .drop("start_geo_hash", "end_geo_hash")

# Дополнительные фильтры для аномалий
print("Применение дополнительных фильтров...")
filtered_df = filtered_df.filter(
    # 1. Удаляем поездки с экстремально малым расстоянием при большой длительности
    ~((col("distance_km") < 0.01) & (col("ride_duration_minutes") > 40)) &
    
    # 2. Фильтр для поездок на ту же станцию с аномальной длительностью
    ~((col("start_station_name") == col("end_station_name")) & 
      (col("ride_duration_minutes") > 1440))
)

# Добавляем скорость и фильтруем
filtered_df = filtered_df.withColumn(
    "speed_kmh",
    when(col("ride_duration_minutes") > 0,
         col("distance_km") / (col("ride_duration_minutes") / 60.0)).otherwise(0)
).filter((col("speed_kmh") <= 45) | (col("speed_kmh").isNull()))

# Финальная очистка
cleaned_df_with_features = filtered_df \
    .withColumn("speed_kmh", 
                when(col("ride_duration_minutes") > 0, 
                     col("distance_km") / (col("ride_duration_minutes") / 60.0)).otherwise(0))

# Кэшируем финальный датафрейм
cleaned_df_with_features.cache()
final_count = cleaned_df_with_features.count()
print(f"✓ ФИНАЛЬНОЕ КОЛИЧЕСТВО ДАННЫХ: {final_count:,} строк")
print(f"✓ ОБЩАЯ ОЧИСТКА: {(initial_count - final_count)/initial_count*100:.2f}% удалено")

# ============================================
# СОЗДАНИЕ ВИТРИН ДАННЫХ
# ============================================

print("\n" + "="*80)
print("СОЗДАНИЕ ВИТРИН ДАННЫХ ДЛЯ UNIT-ЭКОНОМИКИ")
print("="*80)

# Функция для оптимизированного сохранения
def save_dataframe(df, path, name):
    """Сохраняет DataFrame с оптимизацией"""
    print(f"\nСоздание: {name}...")
    
    # Перераспределяем для записи
    df = df.repartition(50)
    
    # Сохраняем с прогрессом
    df.write \
        .mode("overwrite") \
        .option("header", "true") \
        .option("compression", "gzip") \
        .option("encoding", "UTF-8") \
        .option("quoteAll", "true") \
        .option("escape", "\"") \
        .csv(path)
    
    print(f"✓ Успешно сохранено: {name}")

# Создаем папку для результатов
import os
os.makedirs("data", exist_ok=True)

# Витрина 1: Основные метрики поездок
ride_metrics = cleaned_df_with_features.select(
    "ride_id",
    "member_casual",
    "rideable_type",
    "ride_duration_minutes",
    "distance_km",
    "speed_kmh",
    "same_station",
    "start_station_name",
    "end_station_name",
    col("ride_duration_minutes").alias("unit_cost_driver"),
    col("distance_km").alias("unit_distance_driver"),
    when(col("same_station") == True, 1).otherwise(0).alias("is_return_trip")
)

save_dataframe(ride_metrics, "data/ride_metrics.csv", "Основные метрики поездок")

# Витрина 2: Пользовательская активность
user_activity = cleaned_df_with_features.groupBy("member_casual").agg(
    count("*").alias("total_rides"),
    countDistinct("ride_id").alias("unique_users_estimate"),
    round(mean("ride_duration_minutes"), 2).alias("avg_ride_duration"),
    round(mean("distance_km"), 3).alias("avg_ride_distance"),
    round(stddev("ride_duration_minutes"), 2).alias("std_ride_duration"),
    round(sum("ride_duration_minutes") / 60, 1).alias("total_usage_hours"),
    round(mean(when(col("same_station") == True, 1).otherwise(0)) * 100, 1).alias("return_rate_percent")
)

save_dataframe(user_activity, "data/user_activity.csv", "Пользовательская активность")

# Витрина 3: Использование велосипедов
bike_utilization = cleaned_df_with_features.groupBy("rideable_type").agg(
    count("*").alias("total_rides"),
    round(count("*") * 100.0 / cleaned_df_with_features.count(), 2).alias("market_share_percent"),
    round(mean("ride_duration_minutes"), 2).alias("avg_usage_time_min"),
    round(sum("ride_duration_minutes") / 60, 1).alias("total_usage_hours"),
    round(mean("distance_km"), 3).alias("avg_distance_per_ride"),
    round(sum("distance_km"), 1).alias("total_distance_km"),
    round(mean(when(col("same_station") == True, 1).otherwise(0)) * 100, 1).alias("return_rate_percent")
).orderBy(desc("total_rides"))

save_dataframe(bike_utilization, "data/bike_utilization.csv", "Использование велосипедов")

# Витрина 4: Временные паттерны
time_patterns = cleaned_df_with_features.groupBy(
    "ride_year", "ride_month", "ride_dayofweek", "ride_hour"
).agg(
    count("*").alias("ride_count"),
    round(mean("ride_duration_minutes"), 1).alias("avg_duration_min"),
    round(mean("distance_km"), 2).alias("avg_distance_km"),
    round(mean(when(col("member_casual") == "member", 1).otherwise(0)) * 100, 1).alias("member_percent"),
    round(mean(when(col("is_weekend") == True, 1).otherwise(0)) * 100, 1).alias("weekend_percent")
).orderBy("ride_year", "ride_month", "ride_dayofweek", "ride_hour")

save_dataframe(time_patterns, "data/time_patterns.csv", "Временные паттерны")

# Витрина 5: Географическая активность
geo_activity = cleaned_df_with_features.groupBy(
    "start_station_name", "end_station_name"
).agg(
    count("*").alias("route_frequency"),
    round(mean("ride_duration_minutes"), 1).alias("avg_duration_min"),
    round(mean("distance_km"), 2).alias("avg_distance_km"),
    round(stddev("ride_duration_minutes"), 1).alias("std_duration"),
    round(sum(when(col("member_casual") == "member", 1).otherwise(0))).alias("member_count"),
    round(sum(when(col("member_casual") == "casual", 1).otherwise(0))).alias("casual_count")
).filter(col("route_frequency") >= 10)

save_dataframe(geo_activity, "data/geo_activity.csv", "Географическая активность")

# Витрина 6: Станционная экономика
station_economics = cleaned_df_with_features.groupBy("start_station_name").agg(
    count("*").alias("departures_count"),
    round(mean("ride_duration_minutes"), 1).alias("avg_departure_duration"),
    round(mean("distance_km"), 2).alias("avg_departure_distance"),
    countDistinct("end_station_name").alias("unique_destinations"),
    round(sum(when(col("same_station") == True, 1).otherwise(0))).alias("return_count"),
    round(mean(when(col("same_station") == True, 1).otherwise(0)) * 100, 1).alias("return_rate_percent")
).filter(col("departures_count") >= 5).orderBy(desc("departures_count"))

save_dataframe(station_economics, "data/station_economics.csv", "Станционная экономика")

# Витрина 7: Метрики скорости
speed_metrics = cleaned_df_with_features.select(
    "ride_id",
    "member_casual",
    "rideable_type",
    "ride_duration_minutes",
    "distance_km",
    "speed_kmh",
    when(col("speed_kmh") > 25, "high_speed")
    .when(col("speed_kmh") > 15, "medium_speed")
    .otherwise("low_speed").alias("speed_category"),
    when(col("ride_duration_minutes") > 120, "long_ride")
    .when(col("ride_duration_minutes") > 30, "medium_ride")
    .otherwise("short_ride").alias("duration_category")
).filter(col("speed_kmh") > 0)

save_dataframe(speed_metrics, "data/speed_metrics.csv", "Метрики скорости")

# Витрина 8: Экономика возвратов
return_economics = cleaned_df_with_features.groupBy(
    "start_station_name", 
    col("same_station").alias("is_return_trip")
).agg(
    count("*").alias("trip_count"),
    round(mean("ride_duration_minutes"), 1).alias("avg_duration_min"),
    round(mean("distance_km"), 2).alias("avg_distance_km"),
    round(mean(when(col("member_casual") == "member", 1).otherwise(0)) * 100, 1).alias("member_percent")
).orderBy("start_station_name", desc("is_return_trip"))

save_dataframe(return_economics, "data/return_economics.csv", "Экономика возвратов")

# Витрина 9: Агрегированные KPI
aggregated_kpi = cleaned_df_with_features.agg(
    count("*").alias("total_rides"),
    round(mean(when(col("member_casual") == "member", 1).otherwise(0)) * 100, 1).alias("member_percent"),
    round(mean(when(col("member_casual") == "casual", 1).otherwise(0)) * 100, 1).alias("casual_percent"),
    round(mean("ride_duration_minutes"), 1).alias("avg_ride_duration_min"),
    round(mean("distance_km"), 2).alias("avg_ride_distance_km"),
    round(mean("speed_kmh"), 1).alias("avg_speed_kmh"),
    round(mean(when(col("same_station") == True, 1).otherwise(0)) * 100, 1).alias("return_rate_percent"),
    round(sum("ride_duration_minutes") / 60, 0).alias("total_usage_hours"),
    countDistinct("start_station_name").alias("unique_start_stations"),
    countDistinct("end_station_name").alias("unique_end_stations")
)

save_dataframe(aggregated_kpi, "data/aggregated_kpi.csv", "Агрегированные KPI")

# Витрина 10: Детализированные данные
detailed_data = cleaned_df_with_features.select(
    "ride_id",
    "rideable_type",
    "member_casual",
    "started_at",
    "ended_at",
    "ride_duration_minutes",
    "distance_km",
    "speed_kmh",
    "same_station",
    "is_weekend",
    "ride_dayofweek",
    "ride_hour",
    "start_station_name",
    "end_station_name",
    "start_lat",
    "start_lng",
    "end_lat",
    "end_lng",
    (col("ride_duration_minutes") * 0.1).alias("estimated_wear_cost"),
    when(col("same_station") == True, 5.0).otherwise(0.0).alias("estimated_rebalancing_saving")
)

save_dataframe(detailed_data, "data/detailed_data.csv", "Детализированные данные")

# Сохраняем оригинальные данные
print("\n" + "="*80)
print("СОХРАНЕНИЕ ПОЛНОГО НАБОРА ДАННЫХ")
print("="*80)

cleaned_df_with_features.write \
    .mode("overwrite") \
    .option("header", "true") \
    .option("compression", "gzip") \
    .option("encoding", "UTF-8") \
    .csv("data/full_dataset.csv")

print("ПОЛНЫЙ НАБОР ДАННЫХ СОХРАНЕН: data/full_dataset.csv")

# Очистка кэша
cleaned_df_with_features.unpersist()

print("\n" + "="*80)
print("ВСЕ ВИТРИНЫ УСПЕШНО СОЗДАНЫ И СОХРАНЕНЫ!")
print("="*80)
spark.stop()

SPARK СЕССИЯ СОЗДАНА ДЛЯ ОБРАБОТКИ 31 МЛН СТРОК

НАЧАЛО ЗАГРУЗКИ ДАННЫХ ИЗ output.csv
✓ УСПЕШНО ЗАГРУЖЕНО: 31,013,853 строк
✓ КОЛИЧЕСТВО СТОЛБЦОВ: 13
✓ РАЗМЕР ДАННЫХ В ПАМЯТИ: ~3.00 GB (примерно)

СОЗДАНИЕ ПРИЗНАКОВ И ПРЕОБРАЗОВАНИЯ

ПРИМЕНЕНИЕ ФИЛЬТРОВ ДЛЯ ОЧИСТКИ ДАННЫХ
Начальное количество строк: 31,013,853
✓ После фильтрации: 17,137,026 строк
✓ Удалено строк: 13,876,827 (44.74%)

ДОПОЛНИТЕЛЬНАЯ ОЧИСТКА И ЗАПОЛНЕНИЕ ДАННЫХ
Заполнение пропущенных станций...
Применение дополнительных фильтров...
✓ ФИНАЛЬНОЕ КОЛИЧЕСТВО ДАННЫХ: 16,774,028 строк
✓ ОБЩАЯ ОЧИСТКА: 45.91% удалено

СОЗДАНИЕ ВИТРИН ДАННЫХ ДЛЯ UNIT-ЭКОНОМИКИ

Создание: Основные метрики поездок...
✓ Успешно сохранено: Основные метрики поездок

Создание: Пользовательская активность...
✓ Успешно сохранено: Пользовательская активность

Создание: Использование велосипедов...
✓ Успешно сохранено: Использование велосипедов

Создание: Временные паттерны...
✓ Успешно сохранено: Временные паттерны

Создание: Географическая активность...